# © Artur Czarnecki. All rights reserved.
# Integrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [ ]:
from intergrax.globals.settings import GlobalSettings
from intergrax.llm_adapters.llm_provider import LLMAdapterRegistry, LLMProvider
from intergrax.llm_adapters.llm_usage_track import LLMUsageTracker
import intergrax.system_prompts as prompts
from intergrax.rag.embedding_manager import EmbeddingManager
from intergrax.rag.rag_retriever import RagRetriever
from intergrax.rag.rag_answerer import RagAnswerer, AnswererConfig
from intergrax.memory.conversational_memory import ConversationalMemory
from intergrax.tools.tools_base import ToolRegistry, ToolBase
from intergrax.rag.vectorstore_manager import VSConfig, VectorstoreManager
from intergrax.rag.re_ranker import ReRankerConfig, ReRanker
from intergrax.chat_agent import ChatAgentConfig, RagComponent, ChatAgent
from pydantic import BaseModel, Field



# -----------------------------------------------------------
# 1) LLM + conversational memory
# -----------------------------------------------------------
llm = LLMAdapterRegistry.create(LLMProvider.OLLAMA)
memory = ConversationalMemory()


# -----------------------------------------------------------
# 2) Example Tool: Weather
# -----------------------------------------------------------
class WeatherArgs(BaseModel):
    """Arguments required to call the weather tool."""
    city: str = Field(..., description="City name. Example: 'Warsaw'.")


class WeatherTool(ToolBase):
    """
    Demo weather tool that simulates a weather API response.

    Used by the agent when the user requests weather information.
    """
    name = "get_weather"
    description = (
        "Returns the current weather for a given city. Use only for queries "
        "related to weather, temperature or atmospheric conditions."
    )
    schema_model = WeatherArgs

    def run(self, **kwargs):
        city = kwargs["city"]
        # Static demo response.
        return {"city": city, "tempC": 12.3, "summary": "Partly cloudy"}


# Register available tools
tools = ToolRegistry()
tools.register(WeatherTool())


# -----------------------------------------------------------
# 3) Vector store & RAG configuration
# -----------------------------------------------------------
cfg = VSConfig(
    provider="chroma",
    collection_name="intergrax_docs",
    chroma_persist_directory="chroma_db/intergrax_docs_v1",
)

store = VectorstoreManager(config=cfg, verbose=False)

embed_manager = EmbeddingManager(
    verbose=False,
    provider="ollama",
    model_name=GlobalSettings.default_ollama_embed_model,
    assume_ollama_dim=1536,
)

reranker = ReRanker(
    embedding_manager=embed_manager,
    config=ReRankerConfig(
        use_score_fusion=True,
        fusion_alpha=0.4,
        normalize="minmax",
        doc_batch_size=256,
    ),
    verbose=True,
)

retriever = RagRetriever(store, embed_manager, verbose=False)


# -----------------------------------------------------------
# 4) RAG Answerer configuration
# -----------------------------------------------------------
cfg = AnswererConfig(
    top_k=10,
    min_score=0.15,
    re_rank_k=5,
    max_context_chars=12000,
)

cfg.system_instructions = prompts.default_rag_system_instruction()
cfg.system_context_template = (
    "Use the following retrieved context to answer the user question: {context}"
)

answerer_docs = RagAnswerer(
    retriever=retriever,
    llm=llm,
    reranker=reranker,
    config=cfg,
    memory=None,
    verbose=False,
)

# RAG routing logic description
rag_docs = RagComponent(
    name="intergrax_docs",
    answerer=answerer_docs,
    description=(
        "This component responds to questions related to Mooff regulations, "
        "privacy policies, internal documentation, and compliance rules. "
        "Use this component for legally-bound questions or factual document-based queries."
    ),
    priority=10,
)

usage_tracker = LLMUsageTracker(run_id="test")
usage_tracker.register_adapter(llm)

# -----------------------------------------------------------
# 5) High-level hybrid agent (RAG + tools + LLM chat)
# -----------------------------------------------------------
agent = ChatAgent(
    llm=llm,
    memory=memory,
    tools=tools,
    rag_components=[rag_docs],
    config=ChatAgentConfig(verbose=True),
)


# -----------------------------------------------------------
# 6) Test Questions
# -----------------------------------------------------------
for question in [
    "What is the weather in Warsaw?",
    "Explain in detail what Mooff virtual fairs are.",
    "Calculate 22 * 3 and provide just the result.",
]:
    print(question)
    
    response = agent.run(question=question, run_id=usage_tracker.run_id)
    
    print("Routing Decision: ", response["route"])
    print("RAG Component: ", response["rag_component"])
    print("Tool Triggered: ", response["tool_traces"])
    print("Final Answer: ", response["answer"])
    print("=============================================")



What is the weather in Warsaw?
[intergraxChatAgent] ROUTE=tools RAG=-
[intergraxToolsAgent] Iteration 1 (planner)
[intergraxToolsAgent] Calling tool: get_weather({'city': 'Warsaw'})
[intergraxToolsAgent] Iteration 2 (planner)
Routing Decision:  tools
RAG Component:  None
Tool Triggered:  [{'tool': 'get_weather', 'args': {'city': 'Warsaw'}, 'output_preview': '{"city": "Warsaw", "tempC": 12.3, "summary": "Partly cloudy"}', 'output': {'city': 'Warsaw', 'tempC': 12.3, 'summary': 'Partly cloudy'}}]
Final Answer:  The current weather in Warsaw is Partly cloudy with a temperature of 12.3 degrees Celsius.
Explain in detail what Mooff virtual fairs are.
[intergraxChatAgent] ROUTE=general RAG=-
Routing Decision:  general
RAG Component:  None
Tool Triggered:  []
Final Answer:  Moof Virtual Fairs are online events that allow exhibitors to showcase their products, services, and experiences in a virtual environment. They are designed to mimic the experience of traditional trade shows and exhibition

In [4]:
from intergrax.llm_adapters.llm_adapter import LLMRunStats


print("==========Adapters==========")
for label in usage_tracker.registered_labels():
    print(label)

print()
print()

print("==========Total==========")
total:LLMRunStats = usage_tracker.total();
print(f"calls: {total.calls}")
print(f"input_tokens: {total.input_tokens}")
print(f"output_tokens: {total.output_tokens}")
print(f"total_tokens: {total.total_tokens}")
print(f"duration_ms: {total.duration_ms}")
print(f"errors: {total.errors}")

==========Adapters==========
950a839e95d14813acb2d185ea3c46b0


==========Total==========
calls: 0
input_tokens: 0
output_tokens: 0
total_tokens: 0
duration_ms: 0
errors: 0
